In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_pickle('data.pkl')
data = data[data.date_block_num > 11]

In [3]:
drop_features = ['date_cat-medium-type_avg_item_cnt_lag_1', 'date_city_avg_item_cnt_lag_2', 'date_shop_avg_item_cnt_lag_6', 'days_in_month', 'delta_price_trend_with_lag_1', 'delta_price_trend_with_lag_4', 'delta_shop_revenue_this_month', 'delta_shop_revenue_this_month_lag_2', 'item_avg_price_during_month_lag_3']
data.drop(drop_features, axis=1, inplace=True)

In [4]:
X = None
Y = None
x = None

def set_X_Y_x():
    global X, Y, x
    X = data[data.date_block_num < 34].drop(['item_cnt_month'], axis=1)
    Y = data[data.date_block_num < 34]['item_cnt_month']
    x = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

set_X_Y_x()

In [5]:
from sklearn.metrics import mean_squared_error

def rmse(y_te, y_p):
    return np.sqrt(mean_squared_error(y_te, y_p))

In [6]:
test_range = [*range(65, 86)]
test_range = [i/100 for i in test_range]

In [7]:
best_score = (0, 10)

for test_this_range in test_range:
    X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                        train_size=test_this_range,
                                                        random_state=42)
    lf = LinearRegression()
    lf.fit(X_train, y_train)
    prediction = lf.predict(X_test).clip(0,20)

    current_rmse = rmse(y_test, prediction)
    print(f"Train size: {test_this_range}\nRMSE: {current_rmse}")

    if current_rmse < best_score[1]:
        best_score = (test_this_range, current_rmse)

Train size: 0.65
RMSE: 0.8003264699129082
Train size: 0.66
RMSE: 0.7991169668641688
Train size: 0.67
RMSE: 0.7988784880630845
Train size: 0.68
RMSE: 0.7991891834691143
Train size: 0.69
RMSE: 0.7989936114849506
Train size: 0.7
RMSE: 0.7985416214062987
Train size: 0.71
RMSE: 0.7986544663216979
Train size: 0.72
RMSE: 0.7982727821238813
Train size: 0.73
RMSE: 0.7985357456187182
Train size: 0.74
RMSE: 0.7995145659000567
Train size: 0.75
RMSE: 0.799170093619113
Train size: 0.76
RMSE: 0.799367018953535
Train size: 0.77
RMSE: 0.7997282758090738
Train size: 0.78
RMSE: 0.8004436102841819
Train size: 0.79
RMSE: 0.801827651375034
Train size: 0.8
RMSE: 0.8015585994871826
Train size: 0.81
RMSE: 0.8040264738604426
Train size: 0.82
RMSE: 0.8039648760209365
Train size: 0.83
RMSE: 0.8021291016899722
Train size: 0.84
RMSE: 0.8017228202225367
Train size: 0.85
RMSE: 0.8000181006640787


In [8]:
print(best_score)

(0.72, 0.7982727821238813)
